<a href="https://colab.research.google.com/github/CristianCosci/BTC_dataset_Generator_glassnode/blob/main/datasetBuilder_glassnode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests

connected = False

if(not connected):
  drive.mount('/content/drive', force_remount= True)
  path = '/content/drive/MyDrive/progettoBTC/'
  connected = True
  f = open("{}secret.txt".format(path))
  API_KEY = f.read().replace("\n", "")
  f.close()

In [53]:
def download_dataset(interval, since, dataset_type, eth_data=False):
  query_total = [] # To store an array of dict, each for every query
  with open(path+dataset_type,'r') as file:
    # reading each line
    for line in file:
        count = 0
        # print(line)
        link = line.split()[0] # The first element of the line is the link for the query
        query = {'link': link} # Put in a dict
        labels = [] # To store the word of the line
        for word in line.split()[1:]: # Split the word of the line
            labels.append(word)
            count += 1
        for i in range(0, count, 2): # Step of 2 on reading, beacause need couple (name in new df, name returned by query)
          query[labels[i]] = labels[i+1]
          # print(query)
        query_total.append(query)
        query = {} # Reset dict
  
  df = pd.DataFrame()
  for i in query_total:
    link = i.pop('link')
    # print(link)
    res = requests.get(link,
    params={'a': 'BTC','s': since,'i': interval, 'api_key': API_KEY})
    foo_df = pd.json_normalize(json.loads(res.text))
    foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
    foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()
    for j in i.keys(): # To add a new column in the real dataset -> using couple (name in new df, name returned by query)
      print(j)
      df[j] = foo_df[i[j]]
  
  if eth_data: # If requested download also eth data
    if dataset_type == 'hourly_data.txt':
      df = download_dataset_eth(df, interval = interval , since = since, dataset_type = 'hourly_data_eth.txt')
    elif dataset_type == 'daily_data.txt':
      download_dataset_eth(df, interval = interval , since = since, dataset_type = 'hourly_data_eth.txt')
  return df


def download_dataset_eth(df, interval , since, dataset_type):
  query_total = [] # To store an array of dict, each for every query
  with open(path+dataset_type,'r') as file:
    # reading each line
    for line in file:
        count = 0
        # print(line)
        link = line.split()[0] # The first element of the line is the link for the query
        query = {'link': link} # Put in a dict
        labels = [] # To store the word of the line
        for word in line.split()[1:]: # Split the word of the line
            labels.append(word)
            count += 1
        for i in range(0, count, 2): # Step of 2 on reading, beacause need couple (name in new df, name returned by query)
          query[labels[i]] = labels[i+1]
          # print(query)
        query_total.append(query)
        query = {} # Reset dict
  
  df = pd.DataFrame()
  for i in query_total:
    link = i.pop('link')
    res = requests.get(link,
    params={'a': 'ETH','s': since,'i': interval, 'api_key': API_KEY})
    foo_df = pd.json_normalize(json.loads(res.text))
    foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
    foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()
    for j in i.keys(): # To add a new column in the real dataset -> using couple (name in new df, name returned by query)
      print(j)
      df[j] = foo_df[i[j]]
  return df

In [ ]:
df_hourly = download_dataset(interval = '1h', since = '1356998400', dataset_type = 'hourly_data.txt', eth_data = True)
# df_daily = download_dataset(interval = '24h', since = '1356998400', dataset_type = 'daily_data.txt', eth_data = True)

In [ ]:
df_hourly.to_csv(path+"BTC_hourly_dataset.csv")
# df_daily.to_csv(path+"BTC_daily_dataset.csv")

In [58]:
df_hourly

In [ ]:
# FOR DEBUG
res = requests.get('https://api.glassnode.com/v1/metrics/protocols/uniswap_volume_sum' ,
    params={'a': 'ETH','s': '1356998400','i': '1h', 'api_key': API_KEY})
foo_df = pd.json_normalize(json.loads(res.text))
print(foo_df)
foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()

In [ ]:
foo_df